# Codigo de Extraccion de datos de Conagua
## Pasos
1-. Data Scraping de .txt de la pagina de Conagua

2-. Convertir .txt a .csv

In [2]:
%pip install urllib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [1]:
import urllib.request
import re
import pandas as pd
import os

# Webscraping

In [4]:
estaciones_scrapeables = [ 
    '25001', '25003', '25009', '25014', '25015', '25019', '25021', '25022', '25023', '25025',
    '25050', '25062', '25064', '25074', '25077', '25078', '25080', '25081', '25087', '25093',
    '25030', '25033', '25036', '25037', '25038', '25041', '25042', '25044', '25045', '25046',
    '25100', '25102', '25103', '25105', '25106', '25107', '25110', '25115', '25119', '25150',
    '25155', '25158', '25161', '25171', '25172', '25173', '25176', '25178', '25183', '25186'
]

estado = 'sin'

carpeta_raw_txt = "./data_conagua_raw_txt/"

for estacion in estaciones_scrapeables:
    url = f"https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Diarios/{estado}/dia{estacion}.txt"
    filename = f"{carpeta_raw_txt}{estacion}.txt"
    
    try:
        urllib.request.urlretrieve(url, filename)
        print(f"Downloaded {filename}")
    except Exception as e:
        print(f"Error downloading {url}: {e}. File does not exist")

Downloaded ./data_conagua_raw_txt/25001.txt
Downloaded ./data_conagua_raw_txt/25003.txt
Downloaded ./data_conagua_raw_txt/25009.txt
Downloaded ./data_conagua_raw_txt/25014.txt
Downloaded ./data_conagua_raw_txt/25015.txt
Downloaded ./data_conagua_raw_txt/25019.txt
Downloaded ./data_conagua_raw_txt/25021.txt
Downloaded ./data_conagua_raw_txt/25022.txt
Downloaded ./data_conagua_raw_txt/25023.txt
Downloaded ./data_conagua_raw_txt/25025.txt
Downloaded ./data_conagua_raw_txt/25050.txt
Downloaded ./data_conagua_raw_txt/25062.txt
Downloaded ./data_conagua_raw_txt/25064.txt
Downloaded ./data_conagua_raw_txt/25074.txt
Downloaded ./data_conagua_raw_txt/25077.txt
Downloaded ./data_conagua_raw_txt/25078.txt
Downloaded ./data_conagua_raw_txt/25080.txt
Downloaded ./data_conagua_raw_txt/25081.txt
Downloaded ./data_conagua_raw_txt/25087.txt
Downloaded ./data_conagua_raw_txt/25093.txt
Downloaded ./data_conagua_raw_txt/25030.txt
Downloaded ./data_conagua_raw_txt/25033.txt
Downloaded ./data_conagua_raw_tx

# Transformacion .txt a .csv

In [5]:

carpeta_raw_csv = "./data_conagua_raw_csv/"

codificacion = "utf-8"

regex = r'\d{4}-\d{2}-\d{2}'

linea_inicio = 25

lineas_objetivo = [10, 11, 12, 13, 14, 16, 17, 18]

for archivo_txt in estaciones_scrapeables:
    archivo_txt_path = f"{carpeta_raw_txt}NR_{archivo_txt}.txt"
    archivo_csv_path = f"{carpeta_raw_csv}NR_{archivo_txt}.csv"
    
    with open(archivo_txt_path, 'r', encoding=codificacion) as file:
        lineas = file.readlines()
    
    print(f"Procesando archivo: {archivo_txt_path}")
    etiquetas = []
    valores = []
    estacion = "UNKNOWN"
    clasificacion = "UNKNOWN"
    for i in lineas_objetivo:
        
        if i < len(lineas) and ':' in lineas[i]:
            clave, valor = map(str.strip, re.split(r':\s*', lineas[i], maxsplit=1))
            valor = valor.replace('�', '').replace('°', '').replace('ｰ', '').replace("ï¿½", "")
            if clave == 'ALTITUD':
                valor = re.sub(r'\s*msnm', '', valor)
            etiquetas.append(clave)
            valores.append(valor)
        
        if "ESTACIÓN" in lineas[i]:  # Detección explícita de la estación
            match = re.search(r'ESTACIÓN\s*:\s*(\d+)', lineas[i])
            if match:
                estacion = match.group(1)
        if "ESTACION" in lineas[i]:
            match = re.search(r'ESTACION\s*:\s*(\d+)', lineas[i])
            if match:
                estacion = match.group(1)
                
    print(f'Estacion detectada: {estacion}')
    
    datos = []
    for linea in lineas[linea_inicio:]:
        if re.match(regex, linea):
            valores_linea = re.split(r'\t+', linea.strip()) if '\t' in linea else re.split(r'\s+', linea.strip())
            if len(valores_linea) >= 5:
                datos.append(valores_linea[:5])
    
    if not datos:
        print("⚠️ Advertencia: No se encontraron datos climáticos en el archivo.")
        continue
    
    columnas = ['FECHA', 'PRECIP', 'EVAP', 'TMAX', 'TMIN'] + etiquetas
    df = pd.DataFrame(datos, columns=columnas[:len(datos[0])])
    
    
    for i, etiqueta in enumerate(etiquetas):
        df[etiqueta] = valores[i] if i < len(valores) else ""
    
    df.replace('Nulo', None, inplace=True)
    for column in ['PRECIP', 'EVAP', 'TMAX', 'TMIN']:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    
    df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d')
    df = df.dropna(subset=['FECHA'])
    df['MONTH'] = df['FECHA'].dt.month
    df['YEAR'] = df['FECHA'].dt.year
    
    df.to_csv(archivo_csv_path, index=False, encoding=codificacion)
    print(f"Archivo CSV guardado como: NR_{archivo_txt}.csv")
    
    

Procesando archivo: ./data_conagua_raw_txt/NR_25001.txt
Estacion detectada: 25001
Archivo CSV guardado como: NR_25001.csv
Procesando archivo: ./data_conagua_raw_txt/NR_25003.txt
Estacion detectada: 25003
Archivo CSV guardado como: NR_25003.csv
Procesando archivo: ./data_conagua_raw_txt/NR_25009.txt
Estacion detectada: 25009
Archivo CSV guardado como: NR_25009.csv
Procesando archivo: ./data_conagua_raw_txt/NR_25014.txt
Estacion detectada: 25014
Archivo CSV guardado como: NR_25014.csv
Procesando archivo: ./data_conagua_raw_txt/NR_25015.txt
Estacion detectada: 25015
Archivo CSV guardado como: NR_25015.csv
Procesando archivo: ./data_conagua_raw_txt/NR_25019.txt
Estacion detectada: 25019
Archivo CSV guardado como: NR_25019.csv
Procesando archivo: ./data_conagua_raw_txt/NR_25021.txt
Estacion detectada: 25021
Archivo CSV guardado como: NR_25021.csv
Procesando archivo: ./data_conagua_raw_txt/NR_25022.txt
Estacion detectada: 25022
Archivo CSV guardado como: NR_25022.csv
Procesando archivo: ./da

# Calculo Umbral
1-.Creacion de csv de huecos

2-.Calculo de fechas faltantes

3-.Calculo de Registros incompletos totales

4-.Porcentaje de umbral

In [6]:
columnas = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']
fecha_col = 'FECHA'

carpeta_salida = "./data_huecos_conagua/"

for archivo_csv in os.listdir(carpeta_raw_csv):
    archivo_csv_path = f"{carpeta_raw_csv}{archivo_csv}"
    archivo_salida = carpeta_salida + archivo_csv.replace('.csv', '_huecos.csv')
    df = pd.read_csv(archivo_csv_path, encoding=codificacion)
    df_temp = df.copy()
    df_temp = df_temp.sort_values(fecha_col)
    
    resultados = []
    
    for col in columnas:
        is_null = df_temp[col].isnull()
        es_inicio = is_null & (~is_null.shift(1, fill_value=False))
        group_id = es_inicio.cumsum()
        
        df_huecos = df_temp[is_null].copy()
        df_huecos['group_id'] = group_id[is_null]
        
        if df_huecos.empty:
            continue

        agg_results = df_huecos.groupby('group_id').agg(
            inicio=(fecha_col, 'min'),
            fin=(fecha_col, 'max')
        )
        
        agg_results['inicio'] = pd.to_datetime(agg_results['inicio'], errors='coerce')
        agg_results['fin'] = pd.to_datetime(agg_results['fin'], errors='coerce')
        # Calcular la duración de los huecos
        agg_results['duracion'] = (agg_results['fin'] - agg_results['inicio']).dt.days + 1
        
        tipo_huecos = []
        for duracion in agg_results['duracion']:
            if duracion <= 3:
                tipo_huecos.append('corto')
            else:
                tipo_huecos.append('largo')
        agg_results['tipo'] = tipo_huecos
        
        agg_results['columna'] = col
        
        resultados.append(agg_results[['columna', 'inicio', 'fin', 'duracion', 'tipo']])
    
    # Guardar resultados en un csv 
    if resultados:
        df_resultados = pd.concat(resultados, ignore_index=True)
    else:
        # Si no hay huecos, el DF de resultados está vacío
        df_resultados = pd.DataFrame(columns=['columna', 'inicio', 'fin', 'duracion', 'tipo'])

    df_resultados.to_csv(f"{archivo_salida}", index=False, encoding=codificacion)
    print(f"Archivo de huecos guardado como: {archivo_salida}")
    
    

Archivo de huecos guardado como: ./data_huecos_conagua/NR_25001_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25003_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25009_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25014_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25015_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25019_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25021_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25022_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25023_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25025_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25030_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25033_huecos.csv
Archivo de huecos guardado como: ./data_huecos_conagua/NR_25036_huecos.csv
Archivo de huecos guardad

In [ ]:
carpeta_raw_csv = "./data_conagua_raw_csv/"
historial_salida = "./data_historial_umbral_conagua/"
codificacion = "utf-8"
fecha_col = 'FECHA'

required_cols = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']

for archivo_csv in os.listdir(carpeta_raw_csv):
    archivo_csv_path = f"{carpeta_raw_csv}{archivo_csv}"
    if not archivo_csv.endswith('.csv') or 'huecos' in archivo_csv.lower():
        continue

    archivo_csv_path = os.path.join(carpeta_raw_csv, archivo_csv)
    archivo_salida_huecos = os.path.join(carpeta_huecos, archivo_csv.replace('.csv', '_huecos.csv'))

    df = pd.read_csv(archivo_csv_path, encoding=codificacion)

    if fecha_col not in df.columns:
        print(f"Saltando {archivo_csv}: no existe columna '{fecha_col}'")
        continue

    df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce')
    if df[fecha_col].isna().all():
        print(f"Saltando {archivo_csv}: todas las fechas son inválidas")
        continue

    fecha_min = df[fecha_col].min()
    fecha_max = df[fecha_col].max()
    if pd.isna(fecha_min) or pd.isna(fecha_max):
        print(f"Saltando {archivo_csv}: fecha mínima/máxima inválida")
        continue

    total_fechas = pd.date_range(start=fecha_min, end=fecha_max, freq='D')
    all_dates = pd.DataFrame({fecha_col: total_fechas})

    # tipos coincidentes -> merge seguro
    df_completo = all_dates.merge(df, on=fecha_col, how='left')
    total_registros_generados = df_completo.shape[0]

    # Fechas completamente faltantes (no había fila original)
    registros_faltantes = df_completo[required_cols].isnull().all(axis=1).sum()

    # Fechas incompletas: al menos una de las columnas requeridas es nula
    registros_incompletos = df_completo[required_cols].isnull().any(axis=1).sum()

    # Leer file de huecos por columna (opcional, no sumarlo a registros_incompletos)
    missing_by_col = {c: 0 for c in required_cols}
    if os.path.exists(archivo_salida_huecos):
        df_huecos = pd.read_csv(archivo_salida_huecos, encoding=codificacion)
        for c in required_cols:
            dur_sum = pd.to_numeric(df_huecos.loc[df_huecos['columna'] == c, 'duracion'], errors='coerce').sum(skipna=True)
            missing_by_col[c] = int(dur_sum) if not pd.isna(dur_sum) else 0

    porcentaje_faltantes = (registros_incompletos / total_registros_generados) * 100 if total_registros_generados > 0 else 0
    porcentaje_existentes = 100 - porcentaje_faltantes

    print(f"Archivo: {archivo_csv}")
    print(f"Total registros generados: {total_registros_generados}")
    print(f"Fechas sin fila original (completamente faltantes): {registros_faltantes}")
    print(f"Fechas con al menos una columna faltante (incompletas): {registros_incompletos}")
    print("Faltantes por columna (desde archivo de huecos):", missing_by_col)
    print(f"Porcentaje incompletos (>=1 valor faltante): {porcentaje_faltantes:.2f}%")
    print(f"Porcentaje registros completos: {porcentaje_existentes:.2f}%")

    # === Guardar historial de umbral por archivo (archivo .txt por cada .csv) ===
    os.makedirs(historial_salida, exist_ok=True)
    historial_path = os.path.join(historial_salida, archivo_csv.replace('.csv', '_historial.txt'))

    resumen_lines = [
        f"Archivo analizado: {archivo_csv}",
        f"Fecha análisis: {pd.Timestamp.now()}",
        f"Total registros generados (rango fechas): {total_registros_generados}",
        f"Fechas sin fila original (completamente faltantes): {registros_faltantes}",
        f"Fechas con al menos una columna faltante (incompletas): {registros_incompletos}",
        f"Porcentaje incompletos (>=1 valor faltante): {porcentaje_faltantes:.2f}%",
        f"Porcentaje registros completos: {porcentaje_existentes:.2f}%",
        "",
        "Faltantes por columna (desde archivo de huecos):"
    ]
    for c, v in missing_by_col.items():
        resumen_lines.append(f"  - {c}: {v} Porcentaje de faltantes: {(v / total_registros_generados * 100) if total_registros_generados > 0 else 0:.2f}%")

    resumen_lines.append("")
    resumen_lines.append("Notas:")
    resumen_lines.append(" - 'Fechas sin fila original' cuenta días sin fila en el CSV original.")
    resumen_lines.append(" - 'Faltantes por columna' viene del archivo de huecos (duraciones por columna).")
    resumen_text = "\n".join(resumen_lines)

    with open(historial_path, "w", encoding="utf-8") as f:
        f.write(resumen_text)

    print(f"Historial guardado en: {historial_path}")

Archivo: NR_25001.csv
Total registros generados: 23496
Fechas sin fila original (completamente faltantes): 2069
Fechas con al menos una columna faltante (incompletas): 2881
Faltantes por columna (desde archivo de huecos): {'PRECIP': 46, 'EVAP': 894, 'TMAX': 15, 'TMIN': 15}
Porcentaje incompletos (>=1 valor faltante): 12.26%
Porcentaje registros completos: 87.74%
Historial guardado en: ./data_historial_umbral_conagua/NR_25001_historial.txt
Archivo: NR_25003.csv
Total registros generados: 22830
Fechas sin fila original (completamente faltantes): 4919
Fechas con al menos una columna faltante (incompletas): 7255
Faltantes por columna (desde archivo de huecos): {'PRECIP': 81, 'EVAP': 4389, 'TMAX': 53, 'TMIN': 53}
Porcentaje incompletos (>=1 valor faltante): 31.78%
Porcentaje registros completos: 68.22%
Historial guardado en: ./data_historial_umbral_conagua/NR_25003_historial.txt
Archivo: NR_25009.csv
Total registros generados: 23192
Fechas sin fila original (completamente faltantes): 2760
F

## Crear tabla de Resumen de Huecos

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
salida_tabla_dir = "./data_Tablas_Resumen_faltante/"
carpeta_huecos = "./data_huecos_conagua/"
codificacion = "utf-8"
fecha_col = 'FECHA'

os.makedirs(salida_tabla_dir, exist_ok=True)

files = [os.path.join(carpeta_huecos, f) for f in os.listdir(carpeta_huecos) if f.lower().endswith('_huecos.csv') or f.lower().endswith('.csv')]
if not files:
    print("No hay archivos de huecos en:", carpeta_huecos)
else:
    dfs = []
    for f in files:
        try:
            dfh = pd.read_csv(f, encoding='utf-8')
            dfs.append(dfh)
        except Exception as e:
            print("Error leyendo", f, e)
    df_all = pd.concat(dfs, ignore_index=True).copy()

    df_all['duracion'] = pd.to_numeric(df_all['duracion'], errors='coerce').fillna(0).astype(int)
    bins = [0, 3, 7, 31, 180, 365, float('inf')]
    labels = ['1-3Dias', '4-7Dias', '8-31Dias', '32-180Dias', '181-365Dias', '365+Dias']
    df_all['rango'] = pd.cut(df_all['duracion'], bins=bins, labels=labels, right=True)


    tabla = df_all.pivot_table(index='columna', columns='rango', values='duracion', aggfunc='count', fill_value=0)


    columnas_esperadas = ['PRECIP', 'EVAP', 'TMIN', 'TMAX']
    for c in columnas_esperadas:
        if c not in tabla.index:
            tabla.loc[c] = 0
    tabla = tabla.reindex(index=columnas_esperadas)

    for lbl in labels:
        if lbl not in tabla.columns:
            tabla[lbl] = 0
    tabla = tabla[labels]


    tabla.index.name = 'Tipo/Hueco'
    salida_csv = os.path.join(salida_tabla_dir, "resumen_huecos_por_duracion.csv")
    tabla.reset_index().to_csv(salida_csv, index=False, encoding='utf-8')
    print("Resumen guardado en:", salida_csv)


    fig, ax = plt.subplots(figsize=(10,4))
    sns.heatmap(tabla.astype(int), annot=True, fmt='d', cmap='YlOrRd', cbar_kws={'label':'Instancias'}, ax=ax)
    ax.set_title('Conteo de huecos por duración y variable')
    ax.set_ylabel('Variable')
    ax.set_xlabel('Duración')
    plt.tight_layout()
    salida_png = os.path.join(salida_tabla_dir, "resumen_huecos_por_duracion.png")
    fig.savefig(salida_png, dpi=150)
    plt.close(fig)
    print("Gráfica guardada en:", salida_png)


C:\Users\Sarah\AppData\Local\Temp\ipykernel_4196\1470360342.py:32: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tabla = df_all.pivot_table(index='columna', columns='rango', values='duracion', aggfunc='count', fill_value=0)


Resumen guardado en: ./data_Tablas_Resumen_faltante/resumen_huecos_por_duracion.csv
Gráfica guardada en: ./data_Tablas_Resumen_faltante/resumen_huecos_por_duracion.png


### Identificar Archivos por encima del umbral aceptado

In [ ]:
umbral_aceptable = 5.0  # Porcentaje máximo aceptable de datos faltantes
carpeta_historial = "./data_historial_umbral_conagua/"
archivos_aceptables = []
for archivo_historial in os.listdir(carpeta_historial):
    archivo_historial_path = os.path.join(carpeta_historial, archivo_historial)
    with open(archivo_historial_path, "r", encoding="utf-8") as f:
        contenido = f.read()
    
    match = re.search(r"Porcentaje incompletos \(>=1 valor faltante\): ([\d\.]+)%", contenido)
    if match:
        porcentaje_faltantes = float(match.group(1))
        if porcentaje_faltantes > umbral_aceptable:
            print(f"⚠️ Advertencia: El archivo {archivo_historial} tiene un {porcentaje_faltantes:.2f}% de datos faltantes, que excede el umbral aceptable del {umbral_aceptable}%.")
        else:
            print(f"✅ El archivo {archivo_historial} cumple con el umbral aceptable de datos faltantes ({porcentaje_faltantes:.2f}%).")
            archivo_aceptable_raw = archivo_historial.replace('_historial.txt', '.csv')
            archivos_aceptables.append(archivo_aceptable_raw)
    else:
        print(f"❓ No se pudo encontrar el porcentaje de datos faltantes en {archivo_historial}.")

⚠️ Advertencia: El archivo NR_25001_historial.txt tiene un 12.26% de datos faltantes, que excede el umbral aceptable del 10.0%.
⚠️ Advertencia: El archivo NR_25003_historial.txt tiene un 31.78% de datos faltantes, que excede el umbral aceptable del 10.0%.
⚠️ Advertencia: El archivo NR_25009_historial.txt tiene un 23.14% de datos faltantes, que excede el umbral aceptable del 10.0%.
⚠️ Advertencia: El archivo NR_25014_historial.txt tiene un 71.51% de datos faltantes, que excede el umbral aceptable del 10.0%.
✅ El archivo NR_25015_historial.txt cumple con el umbral aceptable de datos faltantes (6.07%).
⚠️ Advertencia: El archivo NR_25019_historial.txt tiene un 14.91% de datos faltantes, que excede el umbral aceptable del 10.0%.
⚠️ Advertencia: El archivo NR_25021_historial.txt tiene un 99.81% de datos faltantes, que excede el umbral aceptable del 10.0%.
⚠️ Advertencia: El archivo NR_25022_historial.txt tiene un 35.93% de datos faltantes, que excede el umbral aceptable del 10.0%.
⚠️ Advert

### Crear copia del archivos aceptables en nueva carpeta

In [20]:
directorio_aceptables = "./data_conagua_aceptables/"
carpeta_raw_csv = "./data_conagua_raw_csv"
os.makedirs(directorio_aceptables, exist_ok=True)

for archivo_aceptable in archivos_aceptables:
    archivo_aceptable_path = os.path.join(carpeta_raw_csv, archivo_aceptable)
    with open(archivo_aceptable_path, "r", encoding="utf-8") as f:
        contenido = f.read()
    with open(os.path.join(directorio_aceptables, archivo_aceptable), "w", encoding="utf-8") as f:
        f.write(contenido)

### Agrupar archivos aceptables acorde a la clasificacion manual

In [21]:
directorio_clasificados = "./data_conagua_clasificada/"
directorio_aceptables = "./data_conagua_aceptables/"
os.makedirs(directorio_clasificados, exist_ok=True)
subcarpetas = ['Tropical', 'Seco', 'Templado']
codificacion = "utf-8"

with open("./clasificaciones_ecosistemas.txt", "r", encoding=codificacion) as f:
        contenido = f.read()

for subcarpeta in subcarpetas:
    os.makedirs(os.path.join(directorio_clasificados, subcarpeta), exist_ok=True)

for archivo_aceptable in os.listdir(directorio_aceptables):
    
    df = pd.read_csv(os.path.join(directorio_aceptables, archivo_aceptable), encoding=codificacion)
    estacion = df['ESTACIÓN'].astype(str).str.strip().iloc[0]
    clasificacion="Unknown"

    pattern = rf'^\s*{re.escape(str(estacion))}\s*=.*$'
    found_line = None
    for line in contenido.splitlines():
        if re.match(pattern, line):
            found_line = line
            break
        
    if found_line:
        part = found_line.split('=')[-1].strip()
        m = re.search(r':\s*([A-Za-zÁÉÍÓÚÑáéíóúñ]+)', part)
        if m:
            primary = m.group(1).upper()
            mapping = {'SECO': 'Seco', 'TROPICAL': 'Tropical', 'TEMPLADO': 'Templado'}
            clasificacion = mapping.get(primary, primary.capitalize())

    df['CLASIFICACION'] = clasificacion
    directorio_clasificado = os.path.join(directorio_clasificados,clasificacion)
    archivo_clasificado = os.path.join(directorio_clasificado,archivo_aceptable)
    df.to_csv(archivo_clasificado, index=False, encoding=codificacion)
    print(f"{estacion} -> {clasificacion}")
    
    
        
        
    


25015 -> Seco
25033 -> Templado
25037 -> Seco
25044 -> Seco
25102 -> Seco


# Combinar dataset por clasificacion

In [31]:
import glob

directorio_clasificado = "./data_conagua_clasificada/"
directorio_salida = "./data_conagua_concatenados/"
codificacion = "utf-8"
os.makedirs(directorio_salida, exist_ok=True)

# Concatenar por cada subcarpeta (Tropical, Seco, Templado, etc.)
for sub in os.listdir(directorio_clasificado):
    subpath = os.path.join(directorio_clasificado, sub)
    if not os.path.isdir(subpath):
        continue

    csv_paths = glob.glob(os.path.join(subpath, "*.csv"))
    if not csv_paths:
        print(f"No hay CSV en {subpath}, se omite.")
        continue

    dfs = []
    for p in csv_paths:
        try:
            df_tmp = pd.read_csv(p, encoding=codificacion)
            # asegurar columna ESTACIÓN como string (si existe)
            if 'ESTACIÓN' in df_tmp.columns:
                df_tmp['ESTACIÓN'] = df_tmp['ESTACIÓN'].astype(str).str.strip()
            dfs.append(df_tmp)
        except Exception as e:
            print(f"Error leyendo {p}: {e}")

    if not dfs:
        print(f"Ningún CSV válido para {sub}")
        continue

    # concat tolerante a columnas distintas
    df_concat = pd.concat(dfs, ignore_index=True, sort=False)

    # Asegurar columna FECHA como datetime y ordenar por fecha antes de guardar
    if 'FECHA' in df_concat.columns:
        df_concat['FECHA'] = pd.to_datetime(df_concat['FECHA'], errors='coerce')
        df_concat = df_concat.sort_values('FECHA').reset_index(drop=True)
    else:
        print(f"Advertencia: 'FECHA' no encontrada en archivos de {sub}; no se ordenará.")

    salida = os.path.join(directorio_salida, f"merged_{sub}.csv")
    df_concat.to_csv(salida, index=False, encoding=codificacion)
    print(f"Concatenados {len(dfs)} archivos en: {salida}")

Concatenados 4 archivos en: ./data_conagua_concatenados/merged_Seco.csv
Concatenados 1 archivos en: ./data_conagua_concatenados/merged_Templado.csv
No hay CSV en ./data_conagua_clasificada/Tropical, se omite.


## Preprocesar archivo concatenado

In [32]:
directorio_concatenados = "./data_conagua_concatenados/"
directorio_preprocesados = "./data_conagua_preprocesados/"
codificacion = "utf-8"
os.makedirs(directorio_preprocesados, exist_ok=True)

for archivo_concatenado in os.listdir(directorio_concatenados):
    archivo_concatenado_path = os.path.join(directorio_concatenados, archivo_concatenado)
    df = pd.read_csv(archivo_concatenado_path, encoding=codificacion)

    if 'FECHA' in df.columns:
        df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
    

    antes = len(df)
    df = df.dropna(how='any')
    eliminadas = antes - len(df)

    if 'FECHA' in df.columns:
        df = df.sort_values('FECHA').reset_index(drop=True)
    
    archivo_preprocesado_path = os.path.join(directorio_preprocesados, archivo_concatenado)
    df.to_csv(archivo_preprocesado_path, index=False, encoding=codificacion)
    print(f"Preprocesado y guardado: {archivo_preprocesado_path} (filas eliminadas: {eliminadas})")

Preprocesado y guardado: ./data_conagua_preprocesados/merged_Seco.csv (filas eliminadas: 2121)
Preprocesado y guardado: ./data_conagua_preprocesados/merged_Templado.csv (filas eliminadas: 549)
